# Session 2

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [8]:
print(tf.__version__)

2.4.3


In [10]:
from pathlib import Path
DATA_PATH = Path("../data/ukdale2.h5").absolute().__str__()

In [11]:
from nilmtk import DataSet
ukdale = DataSet(DATA_PATH)
ukdale.set_window(start='2014-01-01', end='2014-07-01')

In [12]:
ukdale.buildings

OrderedDict([(1, Building(instance=1, dataset='UK-DALE')),
             (2, Building(instance=2, dataset='UK-DALE')),
             (3, Building(instance=3, dataset='UK-DALE')),
             (4, Building(instance=4, dataset='UK-DALE')),
             (5, Building(instance=5, dataset='UK-DALE'))])

In [ ]:
ukdale.buildings[1].elec

In [ ]:
ukdale.buildings[1].elec.proportion_of_energy_submetered()

In [ ]:
ukdale.buildings[1].elec.submeters().energy_per_meter()

In [ ]:
fraction = ukdale.buildings[1].elec.submeters().fraction_per_meter().dropna()
# Create convenient labels
labels = ukdale.buildings[1].elec.get_labels(fraction.index)
plt.figure(figsize=(10,30))
fraction.plot(kind='pie', labels=labels)

In [ ]:
ukdale.buildings[1].elec.plot_when_on(on_power_threshold = 40)

In [ ]:
ukdale.buildings[1].elec['fridge'].good_sections(full_results=True).plot()

In [ ]:
import gc
gc.collect()

## Fridge Detection

In [ ]:
fridge = next(ukdale.buildings[1].elec['fridge'].load(sample_period=30))
aggregate = next(ukdale.buildings[1].elec.mains().load(sample_period=30))

In [ ]:
aggregate.head(), aggregate.shape

In [ ]:
fridge.head(), fridge.shape

In [ ]:
aggregate = aggregate['power']['active']
aggregate.head(n=5)

In [ ]:
def normalise(df):
    """
    Normalises the values in df
    """
    mean = df.fillna(method='ffill').values.mean()
    std = df.fillna(method = 'ffill').values.std()
    return mean, std, (df.fillna(method='ffill').values-mean)/std

mean_agg, std_agg, aggregate = normalise(aggregate)
mean_frz, std_frz, fridge = normalise(fridge)

In [ ]:
fridge.shape, aggregate.shape

In [ ]:
fridge = fridge.reshape(-1)

In [ ]:
# Padding is done here, note other approaches for the padding can be used
WINDOW_SIZE =99
aggregate = np.pad(aggregate, (WINDOW_SIZE//2, WINDOW_SIZE//2 +1))
fridge = np.pad(fridge.reshape(-1), (WINDOW_SIZE//2, WINDOW_SIZE//2 +1))

In [ ]:
aggregate = np.array([
    aggregate[i:i+WINDOW_SIZE] for i in range(len(aggregate)-WINDOW_SIZE)
])
aggregate.shape

In [ ]:
fridge = np.array([
    fridge[i:i+WINDOW_SIZE] for i in range(len(fridge)-WINDOW_SIZE)
])
fridge.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.dates import DateFormatter
import matplotlib.font_manager
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure

plt.rc('text', usetex=False)
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)
plt.rc('axes', labelsize=26)

In [ ]:
index = 300
input_seq = aggregate[index]
output_seq = fridge[index]
figure(figsize=(20,8))
plt.plot(input_seq, label='Input')
plt.plot(output_seq, label='Output')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, Flatten, Dense

def return_seq2seq(sequence_length):
    model = Sequential()
    model.add(Conv1D(30,10,activation="relu",input_shape=(sequence_length,1),strides=2))
    model.add(Conv1D(30, 8, activation='relu', strides=2))
    model.add(Conv1D(40, 6, activation='relu', strides=1))
    model.add(Conv1D(50, 5, activation='relu', strides=1))
    model.add(Dropout(.2))
    model.add(Conv1D(50, 5, activation='relu', strides=1))
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(sequence_length))
    optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='mse', optimizer=optim)
    return model

fridge_model = return_seq2seq(WINDOW_SIZE)

In [ ]:
fridge_model.summary()

In [ ]:
aggregate = np.expand_dims(aggregate, axis=-1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(aggregate, fridge, test_size=.2, shuffle=False)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.shape, y_train.shape

In [ ]:
np.isnan(X_train).sum(), np.isnan(y_train).sum()

In [ ]:
history = fridge_model.fit(X_train, y_train, validation_split=.15, batch_size=64, epochs=3, shuffle=True)

In [ ]:
figure(figsize=(20,8))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.plot()

In [ ]:
y_predict = fridge_model.predict(X_test)

In [ ]:
y_predict.shape

In [ ]:
index = 150
predicted_seq = y_predict[index]
real_seq = y_test[index]
figure(figsize=(20,8))
plt.plot(predicted_seq, label='Predicted')
plt.plot(real_seq, label='Output')
plt.legend()
plt.show()

In [ ]:
def denormlise(array, mean, std):
    return array*std+mean

def aggregate_sequences(prediction):
    l = WINDOW_SIZE
    n = len(prediction) + l - 1
    sum_arr = np.zeros((n))
    counts_arr = np.zeros((n))
    o = len(sum_arr)
    for i in range(len(prediction)):
        sum_arr[i:i + l] += prediction[i].flatten()
        counts_arr[i:i + l] += 1
    for i in range(len(sum_arr)):
        sum_arr[i] = sum_arr[i] / counts_arr[i]
    return denormlise(sum_arr, mean_frz, std_frz)

y_predict = aggregate_sequences(y_predict)
y_real = aggregate_sequences(y_test)

In [ ]:
figure(figsize=(20,8))

plt.plot(y_real, label='Real consumption')
plt.plot(y_predict, label='Predicted consumption')
plt.legend()
plt.plot()

In [ ]:
figure(figsize=(20,8))

plt.plot(y_real[:5000], label='Real consumption')
plt.plot(y_predict[:5000], label='Predicted consumption')
plt.legend()
plt.plot()

In [ ]:
from nilmtk.losses import nde, mae, f1score
print(f"""
- F1-score: {f1score(y_real, y_predict)}
- NDE: {nde(y_real, y_predict)}
- MAE: {mae(y_real, y_predict)}
""")